In [332]:
num_nodes = 6
edges = ((0,1), (0,4), (1,2), (1,3), (1,4), (2,3), (3,4))

In [333]:
import pandas as pd, numpy as np

In [334]:
len(graph.adjacency_list)

5

In [335]:
class Graph:

    def __init__(self, num_nodes, edges):
        self.num_nodes = num_nodes
        self.data = [[] for _ in range(num_nodes)]
        for n1, n2 in edges:
            self.data[n1].append(n2)
            self.data[n2].append(n1)
        # self.adjacency_matrix = self.__adjacency_matrix()

    def __repr__(self):
        return '\n'.join([f"{i}: {j}" for i,j in enumerate(self.data)])

    def __len__(self):
        return len(self.adjacency_list)

    def add_edges(self, *edges):
        for n1, n2 in edges:
            self.data[n1].append(n2)
            self.data[n2].append(n1)

    def remove_edges(self, *edges):
        for n1, n2 in edges:
            self.data[n1].remove(n2)
            self.data[n2].remove(n1)

    @property
    def adjacency_matrix(self):
        arr = np.zeros((self.num_nodes, self.num_nodes))
        arr = np.vectorize(lambda x: int(x))(arr)
        df = pd.DataFrame(arr, columns = range(num_nodes), index = range(num_nodes))
        for n1, N2 in enumerate(self.data):
            for n2 in N2:
                df.loc[n1,n2] = 1
        return df

    @property
    def adjacency_list(self):
        return dict(enumerate(self.data))

In [336]:
graph = Graph(num_nodes, edges)

In [337]:
graph

0: [1, 4]
1: [0, 2, 3, 4]
2: [1, 3]
3: [1, 2, 4]
4: [0, 1, 3]
5: []

In [338]:
graph.adjacency_matrix

,0,1,2,3,4,5
0,0,1,0,0,1,0
1,1,0,1,1,1,0
2,0,1,0,1,0,0
3,0,1,1,0,1,0
4,1,1,0,1,0,0
5,0,0,0,0,0,0


In [339]:
graph.add_edges((0,3))

In [340]:
graph

0: [1, 4, 3]
1: [0, 2, 3, 4]
2: [1, 3]
3: [1, 2, 4, 0]
4: [0, 1, 3]
5: []

In [341]:
graph.adjacency_matrix

,0,1,2,3,4,5
0,0,1,0,1,1,0
1,1,0,1,1,1,0
2,0,1,0,1,0,0
3,1,1,1,0,1,0
4,1,1,0,1,0,0
5,0,0,0,0,0,0


In [342]:
graph.remove_edges((3,0))

In [343]:
graph

0: [1, 4]
1: [0, 2, 3, 4]
2: [1, 3]
3: [1, 2, 4]
4: [0, 1, 3]
5: []

In [344]:
graph.adjacency_matrix

,0,1,2,3,4,5
0,0,1,0,0,1,0
1,1,0,1,1,1,0
2,0,1,0,1,0,0
3,0,1,1,0,1,0
4,1,1,0,1,0,0
5,0,0,0,0,0,0


In [345]:
graph.adjacency_list

{0: [1, 4], 1: [0, 2, 3, 4], 2: [1, 3], 3: [1, 2, 4], 4: [0, 1, 3], 5: []}

In [346]:
len(graph)

6

In [347]:
graph.add_edges((2,5), (3,5))

In [348]:
graph

0: [1, 4]
1: [0, 2, 3, 4]
2: [1, 3, 5]
3: [1, 2, 4, 5]
4: [0, 1, 3]
5: [2, 3]

# Breadth First Search (BFS)

In [349]:
class Queue:

    def __init__(self, *args):
        self.queue = list(args)
        self.generator = self.__iter__()

    def __call__(self):
        return self.queue

    def __len__(self):
        return len(self.queue)

    def __iter__(self):
        for i in self.queue:
            yield i

    def __repr__(self):
        return f"{self.queue}"

    def enqueue(self, i):
        self.queue.append(i)

    def dequeue(self):
        try:
            return next(self.generator)
        except:
            self.generator = self.__iter__()
            return next(self.generator)        

    def pop(self):
        if self.__len__() > 0:
            arr = self.queue.copy()
            self.queue = self.queue[1:]
            return arr[0]

In [351]:
def bfs(graph, root):
    '''
    RETURNS -> node: (distance from root, path parent of node)
    '''
    
    queue = Queue()
    discovered = [False] * len(graph)
    distance = [None] * len(graph)
    parent = [None] * len(graph)

    queue.enqueue(root)
    discovered[root] = True
    distance[root] = 0

    while all(discovered)==False:
        current = queue.dequeue()
        connections = graph.adjacency_list[current]
        for node in connections:
            if discovered[node] == False:
                queue.enqueue(node)
                discovered[node] = True
                distance[node] = 1 + distance[current]
                parent[node] = current
            
    return dict(enumerate(zip(distance,parent)))

In [395]:
b = bfs(graph, 0)

In [396]:
b

{0: (0, None), 1: (1, 0), 2: (2, 1), 3: (2, 1), 4: (1, 0), 5: (3, 2)}

# Depth First Search

In [383]:
class Stack:

    def __init__(self, *args):
        self.stack = list(args)
        self.generator = self.__iter__()

    def __call__(self):
        return self.stack

    def __len__(self):
        return len(self.stack)

    def __iter__(self):
        for i in self.stack[::-1]:
            yield i

    def __repr__(self):
        return f"{self.stack}"

    def push(self, i):
        self.stack.append(i)
        self.generator = self.__iter__()

    def pull(self):
        try:
            return next(self.generator)
        except:
            self.generator = self.__iter__()
            return next(self.generator)        

    def pop(self):
        if self.__len__() > 0:
            arr = self.stack.copy()
            self.queue = self.queue[:-1]
            return arr[-1]

In [407]:
def dfs(graph, root):
    '''
    RETURNS -> node: (distance from root, path parent of node)
    (distance does not matter much here)
    '''   
    stack = Stack()
    discovered = [False] * len(graph)
    distance = [None] * len(graph)
    parent = [None] * len(graph)
    result = []
    
    stack.push(root)
    # discovered[root] = True
    distance[root] = 0

    while all(discovered)==False:
        current = stack.pull()
        discovered[current] = True
        # distance[current] = 
        result.append(current)
        connections = graph.adjacency_list[current]
        for node in connections:
            if discovered[node] == False:
                stack.push(node)
                # discovered[node] = True
                distance[node] = 1 + distance[current]
                parent[node] = current

    return dict(enumerate(zip(distance,parent)))

In [408]:
d = dfs(graph, 0)

In [409]:
d

{0: (0, None), 1: (5, 2), 2: (4, 5), 3: (2, 4), 4: (1, 0), 5: (3, 3)}

In [401]:
b

{0: (0, None), 1: (1, 0), 2: (2, 1), 3: (2, 1), 4: (1, 0), 5: (3, 2)}

# Weighted and Directed Graphs

In [412]:
num_nodes2 = 9
edges2 = ((0,1,3), (0,3,2), (0,8,4), (1,7,4), (2,7,2), (2,3,6), (2,5,1), (3,4,1), (4,8,8), (5,6,8))

In [413]:
num_edges3 = 5
edges3 = ((0,1), (1,2), (2,3), (2,4), (4,2), (3,0))
directed3 = True

In [444]:
class WeightedAndDirectedGraph:

    def __init__(self, num_nodes, edges, weighted=False, directed=False):
        self.num_nodes = num_nodes
        self.weighted = weighted
        self.directed = directed

        self.data = [[] for _ in range(num_nodes)]
        # self.weights = [[] for _ in range(num_nodes)]

        for edge in edges:
            if self.weighted:
                n1, n2, weight = edge
                self.data[n1].append((n2, weight))
                if not self.directed:
                    self.data[n2].append((n1, weight))
            else:
                n1, n2 = edge
                self.data[n1].append(n2)
                if not self.directed:
                    self.data[n2].append(n1)

    def __repr__(self):
        return '\n'.join([f"{i}: {j}" for i,j in enumerate(self.data)])

    def __len__(self):
        return len(self.adjacency_list)

    def add_edges(self, *edges):
        if self.weighted:
            for n1, n2, weight in edges:
                self.data[n1].append((n2, weight))
                if not self.directed:
                    self.data[n2].append((n1, weight))
        else:
            for n1, n2 in edges:
                self.data[n1].append(n2)
                if not self.directed:
                    self.data[n2].apeend(n1)

    def remove_edges(self, *edges):
        if self.weighted:
            for n1, n2, weight in edges:
                self.data[n1].remove((n2, weight))
                if not self.directed:
                    self.data[n2].remove((n1, weight))
        else:
            for n1, n2 in edges:
                self.data[n1].remove(n2)
                if not self.directed:
                    self.data[n2].remove(n1)

    @property
    def adjacency_matrix(self):
        arr = np.zeros((self.num_nodes, self.num_nodes))
        arr = np.vectorize(lambda x: int(x))(arr)
        df = pd.DataFrame(arr, columns = range(self.num_nodes), index = range(self.num_nodes))
        for n1, N2 in enumerate([i for i in self.data if len(i)>0]):
            if self.weighted:
                for n2, weight in N2:
                    df.loc[n1,n2] = weight
            else:
                for n2 in N2:
                    df.loc[n1,n2] = 1
        return df

    @property
    def adjacency_list(self):
        return dict(enumerate(self.data))
                

In [445]:
w = WeightedAndDirectedGraph(num_nodes2, edges2, True, True)

In [446]:
w.adjacency_list

{0: [(1, 3), (3, 2), (8, 4)],
 1: [(7, 4)],
 2: [(7, 2), (3, 6), (5, 1)],
 3: [(4, 1)],
 4: [(8, 8)],
 5: [(6, 8)],
 6: [],
 7: [],
 8: []}

In [447]:
edges2

((0, 1, 3),
 (0, 3, 2),
 (0, 8, 4),
 (1, 7, 4),
 (2, 7, 2),
 (2, 3, 6),
 (2, 5, 1),
 (3, 4, 1),
 (4, 8, 8),
 (5, 6, 8))

In [448]:
 w.adjacency_matrix

,0,1,2,3,4,5,6,7,8
0,0,3,0,2,0,0,0,0,4
1,0,0,0,0,0,0,0,4,0
2,0,0,0,6,0,1,0,2,0
3,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,8
5,0,0,0,0,0,0,8,0,0
6,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0


# Shortest Path

# Dijkstra's algorithm

In [449]:
num_nodes4 = 6
edges4 = ((0,1,4), (0,2,2), (1,2,5), (1,3,10), (2,4,3), (4,3,4), (3,5,11))

In [517]:
def update_distances(graph, current, distance, parent=None):
    '''Update the distances of the current node's neighbour'''
    data = graph.data[current]
    neighbours = [i[0] for i in data]
    weights = [i[1] for i in data]
    for i, node in enumerate(neighbours):
        weight = weights[i]
        if distance[current] + weight < distance[node]:
            distance[node] = distance[current] + weight
            if parent:
                parent[node] = current
    return distance, parent


def pick_nect_node(distance, visited):
    '''Pick the next unvisited node at the smallest distance'''
    min_distance = float('inf')
    min_node = None

    for node in range(len(distance)):
        if (not visited[node]) and (distance[node] < min_distance):
            min_node = node
            min_distance = distance[node]

    return min_node

In [522]:
def shortest_path(weightedGraph, source, target):
    visited = [False] * len(weightedGraph)
    parent = [None] * len(weightedGraph)
    distance = [float('inf')] * len(weightedGraph)
    queue = Queue()

    distance[source] = 0
    queue.enqueue(source)
    idx = 0

    while not visited[target]:
        current = queue.dequeue()
        visited[current] = True
        distance, parent = update_distances(weightedGraph, current, distance, parent)
        next_node = pick_nect_node(distance, visited)
        if next_node:
            queue.enqueue(next_node)
        

    return distance[target], parent

In [523]:
g1 = WeightedAndDirectedGraph(num_nodes4, edges4, True)

In [524]:
g1

0: [(1, 4), (2, 2)]
1: [(0, 4), (2, 5), (3, 10)]
2: [(0, 2), (1, 5), (4, 3)]
3: [(1, 10), (4, 4), (5, 11)]
4: [(2, 3), (3, 4)]
5: [(3, 11)]

In [525]:
shortest_path(g1, 0, 5)

(20, [None, 0, 0, 4, 2, 3])

In [526]:
g2 = WeightedAndDirectedGraph(num_nodes2, edges2, True)

In [527]:
g2

0: [(1, 3), (3, 2), (8, 4)]
1: [(0, 3), (7, 4)]
2: [(7, 2), (3, 6), (5, 1)]
3: [(0, 2), (2, 6), (4, 1)]
4: [(3, 1), (8, 8)]
5: [(2, 1), (6, 8)]
6: [(5, 8)]
7: [(1, 4), (2, 2)]
8: [(0, 4), (4, 8)]

In [532]:
shortest_path(g2, 5, 7)

(3, [None, 7, 5, 2, None, None, 5, 2, None])

### 